In [21]:
import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
df=pd.read_csv('fer2013.csv')

In [2]:
df.head

<bound method NDFrame.head of        emotion                                             pixels        Usage
0            0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...     Training
1            0  151 150 147 155 148 133 111 140 170 174 182 15...     Training
2            2  231 212 156 164 174 138 161 173 182 200 106 38...     Training
3            4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...     Training
4            6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...     Training
5            2  55 55 55 55 55 54 60 68 54 85 151 163 170 179 ...     Training
6            4  20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...     Training
7            3  77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...     Training
8            3  85 84 90 121 101 102 133 153 153 169 177 189 1...     Training
9            2  255 254 255 254 254 179 122 107 95 124 149 150...     Training
10           0  30 24 21 23 25 25 49 67 84 103 120 125 130 139...     Training
11           6  39 75 

In [3]:
df.head(5)

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
emotion    35887 non-null int64
pixels     35887 non-null object
Usage      35887 non-null object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB


In [23]:
X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


In [24]:
num_features = 64
num_labels = 7
batch_size = 128
epochs = 45
width, height = 48, 48

X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [25]:
train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)
test_y


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [26]:
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)
X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)
X_test

array([[ 1.6505896 ,  1.7122597 ,  1.774942  , ..., -0.9225547 ,
         0.18117245,  0.80417955],
       [ 0.45219082,  0.84190166,  1.0658671 , ...,  0.73589116,
         0.6640275 ,  0.5656745 ],
       [-0.6116937 ,  0.02127829, -0.6688345 , ..., -0.33572   ,
        -0.35250947, -0.32558146],
       ...,
       [ 1.6628181 ,  1.7246933 ,  1.7876041 , ..., -0.84601104,
        -0.8226578 , -0.87790906],
       [-1.0519217 , -1.1350546 , -1.0486962 , ..., -1.4073311 ,
        -1.3944598 , -1.4051309 ],
       [-0.7095221 , -0.66257447, -0.6941586 , ..., -0.01678811,
         0.6386141 ,  0.80417955]], dtype=float32)

In [27]:
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [28]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=(X_train.shape[1:])))
model.add(Conv2D(64,kernel_size= (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))

In [29]:
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))
model.add(Dropout(0.5))
model

In [30]:
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(num_labels, activation='softmax'))


In [31]:
model.compile(loss=categorical_crossentropy,
              optimizer=Adam(),
              metrics=['accuracy'])

#Training the model
model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)


Epoch 1/45
225/225 [==============================] - 321s 1s/step - loss: 1.7807 - accuracy: 0.2603 - val_loss: 1.7250 - val_accuracy: 0.3140
Epoch 2/45
225/225 [==============================] - 332s 1s/step - loss: 1.5954 - accuracy: 0.3649 - val_loss: 1.4735 - val_accuracy: 0.4260
Epoch 3/45
225/225 [==============================] - 297s 1s/step - loss: 1.4584 - accuracy: 0.4308 - val_loss: 1.3686 - val_accuracy: 0.4723
Epoch 4/45
225/225 [==============================] - 296s 1s/step - loss: 1.3785 - accuracy: 0.4680 - val_loss: 1.2911 - val_accuracy: 0.4870
Epoch 5/45
225/225 [==============================] - 294s 1s/step - loss: 1.3193 - accuracy: 0.4913 - val_loss: 1.2708 - val_accuracy: 0.5104
Epoch 6/45
225/225 [==============================] - 292s 1s/step - loss: 1.2701 - accuracy: 0.5092 - val_loss: 1.2207 - val_accuracy: 0.5297
Epoch 7/45
225/225 [==============================] - 292s 1s/step - loss: 1.2395 - accuracy: 0.5246 - val_loss: 1.2233 - val_accuracy: 0.5308

In [2]:
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer.h5")

NameError: name 'model' is not defined

In [ ]:
import json

In [ ]:
#with open("fer.json") as datafile:
  #  data=json.load(datafile)
  #  dataframe=pd.DataFrame(data)
#pandas.read_json("fer.json")
import os
import cv2
import numpy as np
from keras.models import model_from_json
from keras.preprocessing import image

#load model
model = model_from_json(open("fer.json", "r").read())

In [ ]:
model.load_weights('fer.h5')


face_haar_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')


cap=cv2.VideoCapture(0)


In [ ]:
while True:
    ret,test_img=cap.read()# captures frame and returns boolean value and captured image
    if not ret:
        continue
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    faces_detected = face_haar_cascade.detectMultiScale(gray_img, 1.32, 5)


    for (x,y,w,h) in faces_detected:
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=7)
        roi_gray=gray_img[y:y+w,x:x+h]#cropping region of interest i.e. face area from  image
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = image.img_to_array(roi_gray)
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels /= 255

        predictions = model.predict(img_pixels)

        #find max indexed array
        max_index = np.argmax(predictions[0])

        emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
        predicted_emotion = emotions[max_index]

        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

    resized_img = cv2.resize(test_img, (1000, 700))
    cv2.imshow('Facial emotion analysis ',resized_img)



    if cv2.waitKey(10) == ord('q'):#wait until 'q' key is pressed
        break

cap.release()
cv2.destroyAllWindows